In [1]:
import sys
import json

sys.path.append("..")

import requests

from common.aidevs_authorize import get_task, get_token, send_answer
from common.openai_requests import send_chat_completion
from common.utils import get_message, print_task

In [2]:
token = get_token("knowledge")
task = get_task(token)
print_task(task, 100)

code
0
****************************************************************************************************
msg
I will ask you a question about the exchange rate, the current population or general knowledge.
Decide whether you will take your knowledge from external sources or from the knowledge of the model
****************************************************************************************************
question
podaj populację Francji
****************************************************************************************************
database #1
Currency http://api.nbp.pl/en.html (use table A)
****************************************************************************************************
database #2
Knowledge about countries https://restcountries.com/ - field 'population'
****************************************************************************************************


In [3]:
def get_country_population(country_response):
    return country_response[0]["population"]


def get_exchange_rate(nbp_response):
    return str(nbp_response["rates"][0]["mid"])


def make_knowledge_request(ai_resp: json):
    db_mapper = {
        "exchange rate": {
            "url": "https://api.nbp.pl/api/exchangerates/rates/a/{currency}/?format=json",
            "fn_call": get_exchange_rate,
        },
        "current population": {
            "url": "https://restcountries.com/v3.1/name/{country}?fields=name,population",
            "fn_call": get_country_population,
        },
    }
    category = db_mapper.get(ai_resp["category"])
    url, fn = category["url"].format(**ai_resp), category["fn_call"]
    resp = requests.get(url)
    return fn(json.loads(resp.text))


system_content = """Please classify the question to category: exchange rate|current population|general knowledge. 
Do not answer the question, just classify it to one of the categories:
###
What is the capital city of Poland?
- {"category": "general knowledge"}
What is the population of USA?
- {"category": "current population", "country": "Czech Republic"}
Please provide current GBPPLN rate.
- {"category": "exchange rate", "currency": "GBP"}
"""
question = task["question"]
print(f"Processing question: {question}...")

classification_resp = send_chat_completion(
    model_version="gpt-3.5-turbo",
    system_content=system_content,
    user_content=task["question"],
)
ai_resp = json.loads(get_message(classification_resp))

if "general knowledge" in ai_resp.values():
    answer_response = send_chat_completion(
        model_version="gpt-4",
        system_content="Be ultra concise and provide answer ultra-briefly",
        user_content=question,
    )
    answer_response = get_message(answer_response)
else:
    answer_response = make_knowledge_request(ai_resp)

Processing question: podaj populację Francji...


In [4]:
answer_response

67391582

In [5]:
send_answer(token, answer_response)

'{\n    "code": 0,\n    "msg": "OK",\n    "note": "CORRECT"\n}'